In [1]:
import word_score
text="It was horrible"
score=word_score.get_tuple(text)
score

it {'neg_score': '0', 'pos_score': '0'}
horribl {'neg_score': '0.625', 'pos_score': '0'}


(0.0, 0.62, 0.0)

In [40]:
import csv
rulebase=[]
with open("nominal_rulebase.csv") as csv_file:
    reader=csv.reader(csv_file)
    rulebase.append(next(reader))
    for row in reader:
        c=row[-1]
        
        row=[[float(i) for i in val.split('[')[1].split(']')[0].split(',')] for val in row[:-1]]+[row[-1]]
        rulebase.append(row)
rulebase[:]

[['pos_score', 'neg_score', 'ratio(pos/neg)', 'class'],
 [[0.0, 0.78], [0.0, 0.5], [3.75, 5.0], 'n'],
 [[0.78, 1.56], [0.0, 0.5], [3.75, 5.0], 'n'],
 [[0.78, 1.56], [0.0, 0.5], [3.75, 5.0], 'n'],
 [[0.0, 0.78], [0.0, 0.5], [3.75, 5.0], 'n'],
 [[1.56, 2.34], [0.0, 0.5], [3.75, 5.0], 'n'],
 [[1.56, 2.34], [0.0, 0.5], [3.75, 5.0], 'n'],
 [[2.34, 3.12], [0.0, 0.5], [3.75, 5.0], 'n'],
 [[0.0, 0.78], [0.0, 0.5], [1.25, 2.5], 'b'],
 [[0.0, 0.78], [1.0, 1.5], [1.25, 2.5], 'b'],
 [[0.78, 1.56], [0.5, 1.0], [1.25, 2.5], 'b'],
 [[0.78, 1.56], [0.5, 1.0], [2.5, 3.75], 'b'],
 [[0.0, 0.78], [0.5, 1.0], [0.0, 1.25], 'b'],
 [[0.0, 0.78], [0.5, 1.0], [0.0, 1.25], 'b'],
 [[0.0, 0.78], [0.5, 1.0], [1.25, 2.5], 'b'],
 [[0.0, 0.78], [0.5, 1.0], [0.0, 1.25], 'b'],
 [[0.0, 0.78], [1.5, 2.0], [0.0, 1.25], 'b']]

In [106]:
def get_prob(i,value,t):
    count=0
    hit=0
    #print("\n\n",i,value,t)
    for row in rulebase[1:]:
        #print(row[i],row[-1])
        if row[-1]==t:
            count+=1
            #print('count:',count)
        if row[-1]==t and row[i]==value:
            hit+=1
            #print('hit:',hit)
    #print(round(float(hit/count),2))    
    return round(float(hit/count),2)

In [119]:
def prep_tuple(t):
    result=[]
    for i in range(len(t)):
        values=[row[i] for row in rulebase[1:]]
        for j in values:
            if t[i]>=j[0] and t[i]<=j[1]:
                result.append(j)
                break
    return result

In [124]:
import pprint as pp
def nb(tup):
    targets=set([row[-1] for row in rulebase[1:]])
    target_score={}
    target_prob={}
    print(targets)
    target_column=[row[-1] for row in rulebase[1:]]
    for t in targets:
        target_score[t]=0
        target_prob[t]=float(target_column.count(t)/len(target_column))
    
    probs={}
        
    for i in range(len(rulebase[0][:-1])):
        probs[i]={}
        active=probs[i]
        column=[row[i] for row in rulebase[1:]]
        for value in column:
            if str(value) not in active:
                active[str(value)]={}
            
                for t in targets:
                    active[str(value)][t]=get_prob(i,value,t)
    pp.pprint (probs)        
    
    
    
    print("\n\n",tup)
    tup=prep_tuple(tup)
    print("\n\n",tup)
    for t in targets:
        target_score[t]=target_prob[t]
        for i in range(len(tup)):
            target_score[t]*=probs[i][str(tup[i])][t]
    print(target_score)
    import operator
    output=max(target_score.items(), key=operator.itemgetter(1))[0]
    print(output)
nb(score)

{'b', 'n'}
{0: {'[0.0, 0.78]': {'b': 0.78, 'n': 0.29},
     '[0.78, 1.56]': {'b': 0.22, 'n': 0.29},
     '[1.56, 2.34]': {'b': 0.0, 'n': 0.29},
     '[2.34, 3.12]': {'b': 0.0, 'n': 0.14}},
 1: {'[0.0, 0.5]': {'b': 0.11, 'n': 1.0},
     '[0.5, 1.0]': {'b': 0.67, 'n': 0.0},
     '[1.0, 1.5]': {'b': 0.11, 'n': 0.0},
     '[1.5, 2.0]': {'b': 0.11, 'n': 0.0}},
 2: {'[0.0, 1.25]': {'b': 0.44, 'n': 0.0},
     '[1.25, 2.5]': {'b': 0.44, 'n': 0.0},
     '[2.5, 3.75]': {'b': 0.11, 'n': 0.0},
     '[3.75, 5.0]': {'b': 0.0, 'n': 1.0}}}


 (0.0, 0.62, 0.0)


 [[0.0, 0.78], [0.5, 1.0], [0.0, 1.25]]
{'b': 0.1293435, 'n': 0.0}
b
